In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
# !wget https://huggingface.co/huseinzol05/bpe/resolve/main/ms-en.subwords

In [3]:
from malaya.text.t2t import text_encoder
import malaya

2022-07-04 22:02:50.018541: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/home/ubuntu/tf-nvidia/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
encoder = text_encoder.SubwordTextEncoder('ms-en.subwords')

In [5]:
encoder.decode([25891])

''

In [6]:
encoder.encode('saya')

[197]

In [7]:
encoder.vocab_size

25880

In [8]:
# !~/tf-nvidia/bin/pip3 install fasttext

In [9]:
fast_text = malaya.language_detection.fasttext()

In [10]:
fast_text.predict(['saya suka', 'i like'])

['malay', 'eng']

In [11]:
class Encoder:
    def __init__(self, encoder):
        self.encoder = encoder
        self.vocab_size = encoder.vocab_size

    def encode(self, s):
        s = [self.encoder.encode(s_) for s_ in s]
        s = [i + [1] for i in s]
        return s

    def decode(self, ids, strip_extraneous = False):
        return self.encoder.decode(ids)

In [12]:
s_encoder = Encoder(encoder)

In [13]:
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.utils import registry
from tqdm import tqdm
from glob import glob
import json

In [14]:
augmented = glob('augmented-ms-en-*.json')
augmented

['augmented-ms-en-v2.json',
 'augmented-ms-en-3.json',
 'augmented-ms-en-2.json',
 'augmented-ms-en-test.json',
 'augmented-ms-en-v3.json',
 'augmented-ms-en-1.json']

In [16]:
@registry.register_problem
class Translation(text_problems.Text2TextProblem):
    @property
    def approx_vocab_size(self):
        return encoder.vocab_size

    @property
    def is_generate_per_split(self):
        # generate_data will shard the data into TRAIN and EVAL for us.
        return False

    @property
    def dataset_splits(self):
        return [
            {'split': problem.DatasetSplit.TRAIN, 'shards': 100},
        ]

    def generate_samples(self, data_dir, tmp_dir, dataset_split):

        for file in augmented:
            with open(file) as fopen:
                data = json.load(fopen)

            for i in tqdm(range(len(data['ms']))):
                if len(data['ms'][i]) and len(data['en'][i]):
                    i, o = s_encoder.encode([data['ms'][i], data['en'][i]])
                    yield {'inputs': i, 'targets': o}
                
        with open('train/left.txt') as fopen:
            left = fopen.read().split('\n')
        
        with open('train/right.txt') as fopen:
            right = fopen.read().split('\n')
            
        
        for i in tqdm(range(len(left))):
            if len(left[i]) and len(right[i]):
                lang_left, lang_right = fast_text.predict([left[i], right[i]])
                if lang_left not in ['malay', 'eng']:
                    continue
                if lang_right not in ['eng']:
                    continue
                i, o = s_encoder.encode([left[i], right[i]])
                yield {'inputs': i, 'targets': o}

    def generate_encoded_samples(self, data_dir, tmp_dir, dataset_split):

        generator = self.generate_samples(data_dir, tmp_dir, dataset_split)
        for sample in generator:
            yield sample

In [17]:
import os
import tensorflow as tf

In [18]:
os.system('rm -rf t2t-noisy-ms-en/data')
DATA_DIR = os.path.expanduser('t2t-noisy-ms-en/data')
TMP_DIR = os.path.expanduser('t2t-noisy-ms-en/tmp')

In [19]:
tf.gfile.MakeDirs(DATA_DIR)
tf.gfile.MakeDirs(TMP_DIR)

In [20]:
from tensor2tensor.utils import registry
from tensor2tensor import problems

In [21]:
PROBLEM = 'translation'
t2t_problem = problems.problem(PROBLEM)
t2t_problem.generate_data(DATA_DIR, TMP_DIR)

  0%|                                                                                                                       | 0/84178 [00:00<?, ?it/s]

INFO:tensorflow:Generating case 0.


INFO:tensorflow:Generating case 0.
  5%|████▋                                                                                                   | 15131/335872 [00:01<00:32, 9750.78it/s]

INFO:tensorflow:Generating case 100000.


INFO:tensorflow:Generating case 100000.
 34%|██████████████████████████████████▉                                                                   | 114886/335872 [00:11<00:21, 10517.15it/s]

...

INFO:tensorflow:Generated 4270649 Examples


INFO:tensorflow:Shuffling data...


INFO:tensorflow:Shuffling data...


INFO:tensorflow:Data shuffled.


INFO:tensorflow:Data shuffled.
